In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

# import the dataframe
dataset_ready_select_features =pd.read_csv('df_mtl_features_engineering.csv')

## Split data and feature slection
from sklearn.model_selection import train_test_split

#Defining the independent variables and dependent variables
airbnb_en=dataset_ready_select_features.copy()
airbnb_en = airbnb_en.loc[(airbnb_en['price'] < 900)]
x = airbnb_en.drop(['price'], axis=1)

# use log10 for the price for a good result
y = airbnb_en['price'].values
#y = np.log10(y)

#Getting Test and Training Set
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.1,random_state=353)

selected_feat=['neighbourhood','property_type', 'room_type', 'accommodates', 'bedrooms', 'beds', 'bed_type',
       'availability_365', 'instant_bookable', 'host_is_superhost', 'number_of_reviews', 'review_scores_rating']

x_train=x_train[selected_feat]
x_test =x_test[selected_feat]


# LR Prediction Model
from sklearn.linear_model import LinearRegression

#Prepare a Linear Regression (LR) Model
reg=LinearRegression()
reg.fit(x_train,y_train)

# Saving model to disk
pickle.dump(reg, open('model.pkl','wb')) 

In [21]:
y_pred=reg.predict(x_test)
#df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
# Compare predicted and actual values
#print('Comparison between the 10 real annonces prices and the predicted prices ')
df = pd.DataFrame({'Actual': np.round(y_test, 0), 
                   'Predicted': np.round(y_pred, 0), 'neighbourhood': x_test['neighbourhood'], 'room_type': x_test['room_type']})
df.head(10)
#y_pred

,Actual,Predicted,neighbourhood,room_type
5613,82.0,79.0,17,2
17374,30.0,83.0,19,0
2225,45.0,95.0,14,2
17644,140.0,216.0,23,2
20152,199.0,173.0,20,2
17794,149.0,156.0,15,2
19727,104.0,123.0,15,0
3058,110.0,199.0,6,2
296,80.0,79.0,17,2
3342,45.0,62.0,3,2


In [22]:
from sklearn.metrics import r2_score
from sklearn.metrics import * # importer tout les metrics d'erreurs

print('Results of the evaluation of our LR model with different Measures:')
print('R2 (score):',round(r2_score(y_test,y_pred),2))
print('MAE:', round(mean_absolute_error(y_test, y_pred),2))
print('MSE:', round(mean_squared_error(y_test, y_pred),2))
print('RMSE:', round(np.sqrt(mean_squared_error(y_test, y_pred)),2))
print('Median ABS Error:', round(median_absolute_error(y_test, y_pred),2))

Results of the evaluation of our LR model with different Measures:
R2 (score): 0.42
MAE: 40.69
MSE: 4836.96
RMSE: 69.55
Median ABS Error: 26.84


In [266]:
data_test = pd.DataFrame({'neighbourhood': ['0.935484'], 'room_type': ['-1.528240'], 'availability_365': ['0.496609']})
y_data_pred=reg.predict(data_test)
print('The prediction price is:',np.round(y_data_pred, 0) )

The prediction price is: [50.]


In [267]:
def normalise_predect_feat(features):
    # import the dataframe
    montreal_listing =pd.read_csv('montreal_airbnb.csv')

    # clean Data
    montreal_listing = montreal_listing.drop(['name','id','neighbourhood_group','host_name','last_review'], axis=1)
    montreal_listing.isnull().sum()
    montreal_listing.dropna(how='any',inplace=True)

    #creating a sub-dataframe with no extreme values / less than 400 
    sub_montreal_listing=montreal_listing[montreal_listing.price < 400]

    # Features Engineering
    feature_sub_montreal_listing = sub_montreal_listing.copy()
    feature_sub_montreal_listing.drop(['latitude','longitude'],axis=1,inplace=True)

    feature_sub_montreal_listing

    # Encoding categorical features (proposed 1)
    categorical_features=['room_type', 'neighbourhood']

    for feature in categorical_features:
        labels_ordered=feature_sub_montreal_listing.groupby([feature])['price'].mean().sort_values().index
        labels_ordered={k:i for i,k in enumerate(labels_ordered,0)}
        feature_sub_montreal_listing[feature]=feature_sub_montreal_listing[feature].map(labels_ordered)

    # Normalise Dataframe (proposed 2)
    # add the predict features at the end of data
    # Pass the row elements as key value pairs to append() function 
    feature_sub_montreal_listing = feature_sub_montreal_listing.append({'neighbourhood' : features[0], 'room_type' : features[1], 'availability_365': features[2]} , ignore_index=True)

    feature_scale=[feature for feature in feature_sub_montreal_listing.columns if feature not in ['host_id','price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count']]
    data = pd.DataFrame()
    for feature in feature_scale:
        data[feature] = (feature_sub_montreal_listing[feature] - feature_sub_montreal_listing[feature].mean())/ (feature_sub_montreal_listing[feature].std())
    
    features_data = pd.DataFrame()
    row = len(data)-1
    features_data = data.loc[row,:]
    return(features_data.to_numpy()) # to_numpy(): convert df to array

In [268]:
int_features = [26,1,226]
final_features_1 = np.array(int_features)
final_features_normalise = normalise_predect_feat(final_features_1)
final_features_normalise

array([ 0.57291701, -1.52808199,  0.90910515])

In [269]:
final_features_data = pd.DataFrame()
final_features_data = [np.array(final_features_normalise)]
final_features_data
y_pred=reg.predict(final_features_data)
y_pred

array([49.62399375])

In [270]:
int_features = [26,1,226]

In [271]:
type(int_features)

list

In [4]:
# import the dataframe
import pandas as pd
dataset_ready_select_features =pd.read_csv('df_mtl_features_engineering.csv')

selected_feat=['neighbourhood','property_type', 'room_type', 'accommodates', 'bedrooms', 'beds', 'bed_type',
       'cleaning_fee', 'extra_people', 'availability_365', 'instant_bookable', 
        'host_is_superhost', 'number_of_reviews', 'review_scores_rating']

dataset_ready_select_features[selected_feat]

,neighbourhood,property_type,room_type,accommodates,bedrooms,beds,bed_type,cleaning_fee,extra_people,availability_365,instant_bookable,host_is_superhost,number_of_reviews,review_scores_rating
0,18,2,0,2,1.0,1.0,1,6,8,76,0,1,245,93.0
1,11,0,0,2,1.0,1.0,1,4,2,286,0,1,143,88.0
2,18,6,2,4,1.0,1.0,1,6,3,342,1,2,175,98.0
3,20,0,2,4,1.0,2.0,1,16,1,288,1,2,324,94.0
4,20,2,2,5,2.0,2.0,1,15,3,297,0,2,66,92.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20171,18,0,2,6,3.0,3.0,1,15,4,105,1,2,0,96.0
20172,18,0,2,8,3.0,4.0,1,15,4,105,1,2,0,96.0
20173,18,0,2,10,3.0,5.0,1,15,4,105,1,2,0,96.0
20174,18,0,2,12,4.0,5.0,1,15,4,105,1,2,0,96.0


In [31]:
dataset_ready_select_features.bed_type.value_counts()

1    19963
0      213
Name: bed_type, dtype: int64

In [24]:
# import the dataframe
import pandas as pd
dataset =pd.read_csv('listings.csv')

selected_feat=['neighbourhood','property_type', 'room_type', 'accommodates', 'bedrooms', 'beds', 'bed_type',
       'cleaning_fee', 'extra_people', 'availability_365', 'instant_bookable', 
        'host_is_superhost', 'number_of_reviews', 'review_scores_rating']

dataset[selected_feat]

/home/bilal/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,neighbourhood,property_type,room_type,accommodates,bedrooms,beds,bed_type,cleaning_fee,extra_people,availability_365,instant_bookable,host_is_superhost,number_of_reviews,review_scores_rating
0,Le Plateau-Mont-Royal,House,Private room,2,1.0,1.0,Real Bed,$0.00,$6.00,76,t,f,245,93.0
1,Le Sud-Ouest,Apartment,Private room,2,1.0,1.0,Real Bed,$25.00,$10.00,286,t,f,143,88.0
2,Le Plateau-Mont-Royal,Loft,Entire home/apt,4,1.0,1.0,Real Bed,$0.00,$20.00,342,f,t,175,98.0
3,Downtown Montreal,Apartment,Entire home/apt,4,1.0,2.0,Real Bed,$75.00,$15.00,288,f,t,324,94.0
4,Downtown Montreal,House,Entire home/apt,5,2.0,2.0,Real Bed,$120.00,$20.00,297,t,t,66,92.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20171,Le Plateau-Mont-Royal,Apartment,Entire home/apt,6,3.0,3.0,Real Bed,NaN,$0.00,105,f,t,0,NaN
20172,Le Plateau-Mont-Royal,Apartment,Entire home/apt,8,3.0,4.0,Real Bed,NaN,$0.00,105,f,t,0,NaN
20173,Le Plateau-Mont-Royal,Apartment,Entire home/apt,10,3.0,5.0,Real Bed,NaN,$0.00,105,f,t,0,NaN
20174,Le Plateau-Mont-Royal,Apartment,Entire home/apt,12,4.0,5.0,Real Bed,NaN,$0.00,105,f,t,0,NaN


In [57]:
dataset.host_is_superhost.value_counts()

f    16305
t     3867
Name: host_is_superhost, dtype: int64

In [58]:
dataset_ready_select_features.host_is_superhost.value_counts()

1    16305
2     3867
0        4
Name: host_is_superhost, dtype: int64

In [48]:
dataset.property_type.describe()

count         20176
unique           29
top       Apartment
freq          14815
Name: property_type, dtype: object